In [1]:
import os
import pymysql

class Fruit:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity


class Fruit_Manager:
    def __init__(self):
        # 初始化，建立连接，需提前创建好数据库
        self.connection = pymysql.connect(
            host='localhost',
            user='root',      
            password='123456',
            database='fruit_db', 
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        self.create_table()
    
    # 初始化表
    def create_table(self):
        with self.connection.cursor() as cursor:        # with语句用完后自动关闭游标
            sql = """
            CREATE TABLE IF NOT EXISTS fruits (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL UNIQUE,
                price DECIMAL(10,2) NOT NULL,
                quantity INT NOT NULL
            )
            """
            cursor.execute(sql)
        self.connection.commit()        # 推送

    def add_fruit(self, fruit):
        with self.connection.cursor() as cursor:
            # 首先尝试插入新水果
            insert_sql = "INSERT INTO fruits (name, price, quantity) VALUES (%s, %s, %s)"
            try:
                cursor.execute(insert_sql, (fruit.name, fruit.price, fruit.quantity))
                self.connection.commit()
                return True
            except pymysql.err.IntegrityError:
                # 如果水果已存在，则更新数量
                update_sql = "UPDATE fruits SET quantity = quantity + %s WHERE name = %s"
                cursor.execute(update_sql, (fruit.quantity, fruit.name))
                self.connection.commit()
                print(f"水果 {fruit.name} 已存在，已增加 {fruit.quantity} 个库存")
                return True

    def remove_fruit(self, fruit_name):
        with self.connection.cursor() as cursor:
            sql = "DELETE FROM fruits WHERE name = %s"
            affected_rows = cursor.execute(sql, (fruit_name,))      #cursor.execute() 返回受影响的行数（即被删除的行数）
            self.connection.commit()
            return affected_rows > 0

    def update_fruit(self, fruit_name, new_price, new_quantity):
        with self.connection.cursor() as cursor:
            sql = "UPDATE fruits SET price = %s, quantity = %s WHERE name = %s"
            affected_rows = cursor.execute(sql, (new_price, new_quantity, fruit_name))
            self.connection.commit()
            return affected_rows > 0

    def get_all_fruits(self):
        with self.connection.cursor() as cursor:
            sql = "SELECT name, price, quantity FROM fruits"
            cursor.execute(sql)
            results = cursor.fetchall()     #fetchall()方法返回一个包含所有行的列表，每行是一个字典或元组
            return [Fruit(result['name'], result['price'], result['quantity']) for result in results]
    
    def get_fruit(self, fruit_name):
        with self.connection.cursor() as cursor:
            sql = "SELECT name, price, quantity FROM fruits WHERE name = %s"
            cursor.execute(sql, (fruit_name,))
            result = cursor.fetchone()
            if result:
                return Fruit(result['name'], result['price'], result['quantity'])
            return None

    def sort_by_price(self):
        # 排序现在由SQL完成
        with self.connection.cursor() as cursor:
            sql = "SELECT name, price, quantity FROM fruits ORDER BY price ASC"  # order by __ ACS升序排列，DESC降序排列
            cursor.execute(sql)
            results = cursor.fetchall()
            return [Fruit(result['name'], result['price'], result['quantity']) for result in results]

    def __del__(self):      # 析构器，当对象被垃圾回收时自动调用，以免数据库链接浪费
        self.connection.close()


# 测试
if __name__ == '__main__':
    fruit_manager = Fruit_Manager()
    while True:
        print(f"*************************************水果超市********************************")
        print("1.查询全部水果")
        print("2.添加水果")
        print("3.删除水果")
        print("4.更新水果信息")
        print("5.按价格排序")
        print("6.查询指定水果")
        print("7.退出系统")
        print("*******************************************************************************")
        choice = input("请输入您的选择：")
        if choice == "1":
            print(f"==========查询全部水果信息==========")
            fruits = fruit_manager.get_all_fruits()
            if len(fruits) == 0:
                print("水果超市里还没有水果")
            else:
                for fruit in fruits:
                    print(f"{fruit.name}：{fruit.price}元，{fruit.quantity}个")
        elif choice == "2":
            print(f"==========上架水果==========")
            name = input("请输入水果名称：")
            price = float(input("请输入水果价格："))
            quantity = int(input("请输入水果数量："))
            fruit = Fruit(name, price, quantity)
            if fruit_manager.add_fruit(fruit):
                print(f"{name}水果添加成功")
        elif choice == "3":
            print(f"==========下架水果==========")
            name = input("请输入要删除的水果名称：")
            if fruit_manager.remove_fruit(name):
                print(f"{name}水果删除成功")
            else:
                print(f"{name}水果不存在")
        elif choice == "4":
            print(f"==========更新水果信息==========")
            name = input("请输入要更新的水果名称：")
            new_price = float(input("请输入新的水果价格："))
            new_quantity = int(input("请输入新的水果数量："))
            if fruit_manager.update_fruit(name, new_price, new_quantity):
                print(f"{name}水果信息更新成功")
            else:
                print(f"{name}水果不存在")
        elif choice == "5":
            print(f"==========按价格升序排列==========")
            fruits = fruit_manager.sort_by_price()
            if len(fruits) == 0:
                print("水果超市里还没有水果")
            else:
                print("按照价格排序后的水果信息：")
                for fruit in fruits:
                    print(f"{fruit.name}：{fruit.price}元，{fruit.quantity}个")
        elif choice == "6":
            print(f"==========查询指定水果==========")
            name = input("请输入要查询的水果名称：")
            fruit = fruit_manager.get_fruit(name)
            if fruit is None:
                print(f"{name}水果不存在")
            else:
                print(f"{name}水果信息：{fruit.price}元，{fruit.quantity}个")
        elif choice == "7":
            break
        else:
            print("输入错误，请重新输入")

*************************************水果超市********************************
1.查询全部水果
2.添加水果
3.删除水果
4.更新水果信息
5.按价格排序
6.查询指定水果
7.退出系统
*******************************************************************************
==========查询全部水果信息==========
banana：2.5元，1000.0个
榴莲：50.0元，200.0个
apple：6.0元，300.0个
*************************************水果超市********************************
1.查询全部水果
2.添加水果
3.删除水果
4.更新水果信息
5.按价格排序
6.查询指定水果
7.退出系统
*******************************************************************************
